In [1]:
import logging
import boto3
from botocore.exceptions import ClientError
from dotenv import load_dotenv
import random
import os
import uuid
from pathlib import Path

def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """
    load_dotenv()
    aws_access_key_id = os.getenv("aws_access_key_id")
    aws_secret_access_key = os.getenv("aws_secret_access_key")

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client(
                        "s3",
                        aws_access_key_id=aws_access_key_id,
                        aws_secret_access_key=aws_secret_access_key,
                        )
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client(
                        "s3",
                        aws_access_key_id=aws_access_key_id,
                        aws_secret_access_key=aws_secret_access_key,
                        region_name=region
                        )
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [9]:
# sentinel on aws is in region eu-central-1 therefore we use this region to avoid costs
region = "eu-central-1"
gen_ran_num = random.randrange(100000,999999)
bucket_name = f"solar-detection-{gen_ran_num}-{region}"

In [2]:
bucket_name

NameError: name 'bucket_name' is not defined

In [11]:
create_bucket(bucket_name, region)

True

In [5]:
s3 = boto3.client('s3')
response = s3.list_buckets()

In [13]:
# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  solar-detection-697553-eu-central-1
  solar-detection-ddf3ed03-f48f-11ed-9394-3413e8c504df-eu-west-1


In [15]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [16]:
file_path = Path(r"C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\data_local\test_images_geotiff2\image_0_4.tif")
file_name = open(file_path, 'rb')

In [17]:
upload_file(file_path, bucket_name, object_name='\test\image_0_4.tif')

True

In [8]:
bucket_name = "solar-detection-697553-eu-central-1"

In [14]:
response = s3.list_objects_v2(
  Bucket=bucket_name,
  Prefix='data_raw'
)

In [15]:
response['Contents']

[{'Key': 'data_raw/S2A_MSIL2A_20230513T100551_N0509_R022_T33UVV_20230513T161552/B02_10m.jp2',
  'LastModified': datetime.datetime(2023, 5, 17, 19, 23, 12, tzinfo=tzutc()),
  'ETag': '"1a6b331181c4b5e2410deac64a38ef3f-14"',
  'Size': 111611343,
  'StorageClass': 'STANDARD'},
 {'Key': 'data_raw/S2A_MSIL2A_20230513T100551_N0509_R022_T33UVV_20230513T161552/B03_10m.jp2',
  'LastModified': datetime.datetime(2023, 5, 17, 19, 23, 38, tzinfo=tzutc()),
  'ETag': '"823e7e69c3cf4cf9254ffb5cde7388c4-14"',
  'Size': 114026241,
  'StorageClass': 'STANDARD'},
 {'Key': 'data_raw/S2A_MSIL2A_20230513T100551_N0509_R022_T33UVV_20230513T161552/B04_10m.jp2',
  'LastModified': datetime.datetime(2023, 5, 17, 19, 24, 1, tzinfo=tzutc()),
  'ETag': '"9c9e5b6d6aaad58a4eed98fb999272c2-14"',
  'Size': 113288491,
  'StorageClass': 'STANDARD'},
 {'Key': 'data_raw/S2A_MSIL2A_20230513T100551_N0509_R022_T33UVV_20230513T161552/B08_10m.jp2',
  'LastModified': datetime.datetime(2023, 5, 17, 19, 24, 24, tzinfo=tzutc()),
  'E

In [33]:
try:
    response = s3.head_object(Bucket=bucket_name, Key='\test\image_0_4.tif')
except ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise e

In [29]:
response['ResponseMetadata']['HTTPStatusCode']

200

In [26]:
s3.download_file(bucket_name, '\test\image_0_4.tif', r'C:\Users\Fabian\Downloads\image_0_4.tif')

In [27]:
s3.delete_object(Bucket=bucket_name, Key='\test\image_0_4.tif')

{'ResponseMetadata': {'RequestId': '4V6SWY189J6F6ZFJ',
  'HostId': 'wW4EmoHifbNjjAAQUf/BdRob6+SMCmaLaT+Ogiqset53uATPj4/2IH0L7WvPFEo7HKiophWBFpQ=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'wW4EmoHifbNjjAAQUf/BdRob6+SMCmaLaT+Ogiqset53uATPj4/2IH0L7WvPFEo7HKiophWBFpQ=',
   'x-amz-request-id': '4V6SWY189J6F6ZFJ',
   'date': 'Wed, 17 May 2023 10:03:43 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [28]:
response = s3.list_objects_v2(
  Bucket=bucket_name
)

In [30]:
response

{'ResponseMetadata': {'RequestId': 'DS1BN3MFJTNEFAKT',
  'HostId': '1eHwCUpEOhnRKrWTCUnmdJNIXmlqmJOlxhJaZ1vggiommI81XG++rqRmA6IKKx90MpWdjnr88TM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '1eHwCUpEOhnRKrWTCUnmdJNIXmlqmJOlxhJaZ1vggiommI81XG++rqRmA6IKKx90MpWdjnr88TM=',
   'x-amz-request-id': 'DS1BN3MFJTNEFAKT',
   'date': 'Wed, 17 May 2023 10:03:55 GMT',
   'x-amz-bucket-region': 'eu-west-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Name': 'solar-detection-ddf3ed03-f48f-11ed-9394-3413e8c504df-eu-west-1',
 'Prefix': '',
 'MaxKeys': 1000,
 'EncodingType': 'url',
 'KeyCount': 0}

In [16]:
test = Path()

In [22]:
not test.is_dir()

False

In [23]:
test2 = Path(r"C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\data_local\test_images_geotiff2\image_0_4.tif")
not test2.is_dir()

True